In [101]:
import argparse

# Без этого у меня почему-то падал kernel
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [102]:
# Проверка, что GPU работает
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
# Сет сида, для повторяемости
torch.manual_seed(42)

True
0
NVIDIA GeForce RTX 4060


In [127]:
class MultiLayerPerceptron(nn.Module):

    def __init__(self):
        super(MultiLayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(357, 180)# Это у меня ошибка или я побил датасет?
        self.dropout1 = nn.Dropout(0.5)
        self.bn1 = nn.BatchNorm1d(180)
        self.fc2 = nn.Linear(180, 90)
        self.bn2 = nn.BatchNorm1d(90)
        self.fc3 = nn.Linear(90, 3)

    def forward(self, x: torch.Tensor):
        out1 = torch.relu(self.bn1(self.dropout1(self.fc1(x))))
        out2 = torch.relu(self.bn2(self.fc2(out1)))
        out3 = self.fc3(out2)
        return out3
    
    def predict(self, x: torch.Tensor):
        with torch.no_grad():
            probabilities = torch.softmax(self.forward(x), dim=1)
            predicted_classes = torch.argmax(probabilities, dim=1)
        return predicted_classes

In [128]:
def load_data(train_csv: str, val_csv: str, test_csv: str):

    train_df = pd.read_csv(train_csv)
    val_df = pd.read_csv(val_csv)
    test_df = pd.read_csv(test_csv)

    y_train = train_df["order0"]
    y_val = val_df["order0"]


    X_train = train_df.iloc[:, :-3]
    X_val = val_df.iloc[:, :-3]

    X_test = test_df.iloc[:, :-3]

    return X_train, y_train, X_val, y_val, X_test

In [129]:
def preprocess_data(X_train: pd.DataFrame, y_train: pd.Series, X_val: pd.DataFrame, y_val: pd.Series, X_test: pd.DataFrame):
    # Находим пересечение колонок между всеми датасетами
    common_columns = X_train.columns.intersection(X_val.columns).intersection(X_test.columns)
    
    # Приводим все наборы данных к одинаковым колонкам
    X_train = X_train[common_columns]
    X_val = X_val[common_columns]
    X_test = X_test[common_columns]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    y_train = torch.tensor(y_train.values, dtype=torch.int64)
    X_train = torch.tensor(X_train, dtype=torch.float32)

    y_val = torch.tensor(y_val.values, dtype=torch.int64)
    X_val = torch.tensor(X_val, dtype=torch.float32)

    X_test = torch.tensor(X_test, dtype=torch.float32)

    return X_train, y_train, X_val, y_val, X_test


In [130]:
def evaluate(model: MultiLayerPerceptron, X: torch.Tensor, y: torch.Tensor):
    model.eval()
    predictions = model.predict(X)
    accuracy = accuracy_score(y_true=y.numpy(), y_pred=predictions.numpy())
    conf_matrix = confusion_matrix(y_true=y.numpy(), y_pred=predictions.numpy())
    return predictions, accuracy, conf_matrix

In [131]:
def init_model(learning_rate: float):
    model = MultiLayerPerceptron()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    return model, criterion, optimizer

In [132]:
def train(model: MultiLayerPerceptron, criterion: nn.Module, optimizer: optim.Optimizer, X_train: torch.Tensor, y_train: torch.Tensor, X_val: torch.Tensor, y_val: torch.Tensor, epochs: int, batch_size: int):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for i in range(0, X_train.size(0), batch_size):
            X_batch = X_train[i: i + batch_size]
            y_batch = y_train[i: i + batch_size]

            optimizer.zero_grad()
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= X_train.size(0) // batch_size

        predictions_val, accuracy_val, _ = evaluate(model, X_val, y_val)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss}, Validation Accuracy: {accuracy_val}")

    return model

In [133]:
def main(args: argparse.Namespace):
    X_train, y_train, X_val, y_val, X_test = load_data(args.train_csv, args.val_csv, args.test_csv)
    
    X_train, y_train, X_val, y_val, X_test = preprocess_data(X_train, y_train, X_val, y_val, X_test)

    model, criterion, optimizer = init_model(args.lr)

    train(model, criterion, optimizer, X_train, y_train, X_val, y_val, args.num_epoches, args.batch_size)

    predictions_test = model.predict(X_test).numpy()

    pd.Series(predictions_test).to_csv(args.out_csv, index=False)

In [134]:
if __name__ == '__main__':
    args_dict = {
        'train_csv': './data/train.csv',
        'val_csv': './data/val.csv',
        'test_csv': './data/test.csv',
        'out_csv': './data/submission.csv',
        'lr': 6.106429040577424e-05,
        'batch_size': 1024,
        'num_epoches': 100
    }

    args = argparse.Namespace(**args_dict)
    main(args)

Epoch 1/100, Loss: 1.0792552252446324, Validation Accuracy: 0.5868444220852723
Epoch 2/100, Loss: 0.9769265444810725, Validation Accuracy: 0.6418060621305496
Epoch 3/100, Loss: 0.9072213187690609, Validation Accuracy: 0.6747578920890454
Epoch 4/100, Loss: 0.857970037243583, Validation Accuracy: 0.6933719029052949
Epoch 5/100, Loss: 0.8238932470644801, Validation Accuracy: 0.7016727455665954
Epoch 6/100, Loss: 0.8024621265978853, Validation Accuracy: 0.7089674254810716
Epoch 7/100, Loss: 0.7849979430190788, Validation Accuracy: 0.717268268142372
Epoch 8/100, Loss: 0.7699606349645567, Validation Accuracy: 0.7243114073701421
Epoch 9/100, Loss: 0.7617242700797467, Validation Accuracy: 0.7300968431643818
Epoch 10/100, Loss: 0.7525970241255011, Validation Accuracy: 0.7358822789586216
Epoch 11/100, Loss: 0.7449160425130986, Validation Accuracy: 0.7417934850962143
Epoch 12/100, Loss: 0.737786479232725, Validation Accuracy: 0.7445604326499812
Epoch 13/100, Loss: 0.7312164853427036, Validation A